Convert Natus data to NWB

July 15 2024

Noah Markowitz

In [1]:
from nwreader import read_erd, read_annotations_file

import glob
import os.path as op
import os
import numpy as np
import pandas as pd
import re

from datetime import datetime
from uuid import uuid4
from dateutil.tz import tzlocal

from uuid import uuid4
from hdmf.common.table import DynamicTable
from hdmf.backends.hdf5.h5_utils import H5DataIO
from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.file import Subject, ElectrodeTable
from pynwb.device import Device
from pynwb.ecephys import ElectrodeGroup, ElectricalSeries
import dask.array as da
import h5py

In [2]:
from scripts.nwb import (create_subject, clean_labelfile, create_electrode_groups, 
labelfile2table, create_table_regions, create_electrical_series)
from scripts.utils import load_nwb_settings

# Setup

Specify info on subject and study

In [3]:
# Path to study directory
# NOTE: You should not read in the original study saved on the server.
#       In reading files there's a chance for corruption. Make a copy of it
# Path to w drive is: /run/user/1000/gvfs/smb-share:server=sykisilon1,share=mehtalab/
study_directory = "/media/hbml/HDD2/natus_api/data/natus/amayareyes-jacinto_IC-Day-5A"

# Freesurfer info
freesurfer_subject_directory = "/home/hbml/freesurfer_7.1.1/subjects"
freesurfer_subject_id = "NS144_02"

# NWB filename. Suggested to follow the BIDS template such as: sub-NS001_ses-implant01_desc-day1_ieeg.nwb
nwb_filename = "/media/hbml/HDD2/natus_api/data/nwb/sub-NS144_ses-implant02_desc-ImplantDay5_ieeg.nwb"

# Text fields for NWB file. Leave blank for default to be filled

# Typically the freesurfer and subject ID are the same but not always such as if they had multiple implants

subject_info = {
    "subject_id": "NS144",
    "age": 30, # An integer or ISO 8601 format (ex: 30 years old = "P30Y"). ISO may be good if children to also have months
    "sex": "U", # Options: [M,F,U]
    "species": "Homo sapiens",
    "description": "intracranial patient" # can put another description if you'd like
}

notes = """

"""

session_description = """

"""

session_id = ""


experiment_description = """

"""

experimenter = """

"""

institution = """

"""

lab = """

"""

data_collection = """

"""

# Options: DC1-DC14, TRIG, OSAT, PR, PLETH
other_chans = []


# Fill this dictionary. It will be used for conversion
nwb_info = {
    "notes": notes,
    "session_description": session_description,
    "experiment_description": experiment_description,
    #"experimenter": experimenter,
    "institution": institution,
    "lab": lab,
    "data_collection": data_collection,
    "session_id": session_id,
    #"nwb_filename": nwb_filename
}

# Read the natus study data

save it to a tmp file to later use

In [4]:
erd = read_erd(study_directory, use_dask=True, convert=True, pad_discont=True)

# Sampling rate
fs = erd.attrs["sample_freq"]

# Keep month, date and time but set the year to 1900
study_start_time = erd.attrs["creation_time"].replace(year=1900)

Get the study start time and then change the year

In [5]:
# Time study started. Set Year,Month,Day to 1900,01,01 to annonymize only the day
nwb_info["session_start_time"] = study_start_time

# Read in settings

In [6]:
settings = load_nwb_settings()

# Meta-data, Subject, Device

Fill in meta-data if some are empty

In [7]:
meta_data_settings = settings["meta_data"]
for param, value in nwb_info.items():
    if isinstance(value, datetime):
        continue
    if len(value.strip())==0:
        nwb_info[param] = meta_data_settings[param]

Create subject object

In [8]:
subject_settings = settings["subject"]
for param, value in subject_settings.items():
    if isinstance(subject_info[param], int):
        continue
    if len(subject_info[param])==0:
        subject_info[param] = subject_settings[param]

subject = create_subject(**subject_info)

Create device

In [9]:
amplifier = Device(name="Natus", manufacturer="Natus Medical Inc.", description="Quantum Headbox")

# Read correspondence sheet and make table

In [10]:
# Import correspondence sheet
corr_list = glob.glob(op.join(freesurfer_subject_directory, freesurfer_subject_id, "elec_recon", "*correspondence*"))
corr_file = corr_list[0]
if corr_file.endswith('.csv'):
    corr_df = pd.read_csv(corr_file)
elif corr_file.endswith('.xlsx'):
    corr_df = pd.read_excel(corr_file, sheet_name=0)
else:
    raise ValueError("Correspondence file must be a .csv or .xlsx file")

# Clean correspondence sheet and get more info from it
df, dynamic_columns, neural_data_indices = clean_labelfile(corr_df)

# Group electrodes by name
df, elecgroups = create_electrode_groups(df, amplifier)

# Create DynamicTable for file
electable = labelfile2table(df, dynamic_columns)

# Divide the table into regions to create an ElectricalSeries for each region

In [11]:
# Create the regions
table_regions = create_table_regions(df, electable)

# Create the ElectricalSeries objects
#acquisitions = create_electrical_series(stored_data, table_regions, fs)

Save the subselection of data

In [29]:
tmp_h5_handles = {"filename": [], "file_handle": [], "data_handle":[]}
acquisitions = []
for spec, region in table_regions.items():
    indices = region.data
    data_indices = neural_data_indices[indices].tolist()
    data = erd.data[data_indices,:].T
    tmp_filename = op.join(op.dirname(nwb_filename), spec + ".hdf5")
    print(f"----->Saving {spec} data to temporary hdf5 file. This could take a second")
    da.to_hdf5(tmp_filename, "/data", data, chunks=True, compression="gzip")

    # open the temp file
    f = h5py.File(tmp_filename, "r")
    stored_data = f["/data"] # HDMF5 dataset object
    tmp_h5_handles[spec] = f

    # Create container for the data
    es = ElectricalSeries(
        name=spec,
        data=H5DataIO(data=stored_data, link_data=False),
        electrodes=region,
        rate=float(fs),
        description=spec + " data",
        starting_time=0.0,
        conversion=1e-6,
        #unit="microVolt",
    )
    acquisitions.append(es)

    tmp_h5_handles["filename"].append(tmp_filename)
    tmp_h5_handles["file_handle"].append(f)
    tmp_h5_handles["data_handle"].append(stored_data)

----->Saving ieeg data to temporary hdf5 file. This could take a second
----->Saving epicranial data to temporary hdf5 file. This could take a second


# Add other TimeSeries if specified

# Create NwbFile object then write

In [30]:
# Create the object
nwbfile = NWBFile(
    **nwb_info,
    subject=subject,
    devices=[amplifier],
    acquisition=acquisitions,
    electrodes=electable,
    electrode_groups=elecgroups,
    identifier=str(uuid4()),
)

# Write it
with NWBHDF5IO(nwb_filename, "w") as io:
    io.write(nwbfile,link_data=False)

In [33]:
# Close all temp file
n_files = len(tmp_h5_handles["filename"])
for ii in range(n_files):
    tmp_h5_handles["file_handle"][ii].close()
    os.remove(tmp_h5_handles["filename"][ii])